# Bločne šifre

Cilji laboratorijske vaje so sledeči:
- TODO

## Ustvarjanje pravih naključnih vrednosti

Naključne vrednosti bomo v Pythonu ustvarili s pomočjo modula `secrets`. [Povezava na dokumentacijo.](https://docs.python.org/3/library/secrets.html)

Najprej modul uvozimo, nato pa imlementiramo funkcijo, katera ustvari seznam naključnih bajtov za podano dolžino.

In [1]:
import secrets, os

Npr. pridobimo 16 bajtov za simetrični ključ ali za vrednost NONCE ali IV.

## Naloga 1: Šifriranje s AES-CTR

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

In [14]:
def enc_aes_ctr(k, pt, iv=None):
    """Šifrira čistopis s ključem.
    
    Če IV ni podan, ustvari naključnega.
    Kot rezultat vrne par (IV, tajnopis)."""
    
    if not iv:
        iv = secrets.token_bytes(16)
    
    cipher = Cipher(algorithms.AES(k), modes.CTR(iv))
    encryptor = cipher.encryptor()
    ct = encryptor.update(pt) + encryptor.finalize()

    return iv, ct

In [4]:
def test_enc_aes_ctr():   
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    
    msg = "Hello World!".encode("utf8")
    iv, ct = enc_aes_ctr(key, msg, iv)

    assert iv == bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    assert ct == bytes.fromhex("aa9169004da9689efa74f00d")

test_enc_aes_ctr()

In [11]:
def dec_aes_ctr(k, ct, iv):
    """Dešifrira čistopis s ključem in vrednostjo IV"""
    
    decryptor = Cipher(
        algorithms.AES(k),
        modes.CTR(iv)
    ).decryptor()
    
    return decryptor.update(ct) + decryptor.finalize()

In [6]:
def test_dec_aes_ctr():  
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    ct = bytes.fromhex("aa9169004da9689efa74f00d")
    
    pt = dec_aes_ctr(key, ct, iv)

    assert pt == b"Hello World!"

test_dec_aes_ctr()

## Naloga 2: Šifriranje s AES-CBC

In [12]:
def enc_aes_cbc(k, pt, iv):
    """Šifrira čistopis s ključem.
    
    Če IV ni podan, ustvari naključnega.
    Kot rezultat vrne par (IV, tajnopis)."""
    
    cipher = Cipher(algorithms.AES(k), modes.CBC(iv))
    encryptor = cipher.encryptor()
    
    padder = padding.PKCS7(128).padder()
    pt = padder.update(pt) + padder.finalize()
    
    ct = encryptor.update(pt) + encryptor.finalize()

    return iv, ct

In [13]:
def test_enc_aes_cbc():   
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    
    msg = "Hello World!".encode("utf8")
    iv, ct = enc_aes_cbc(key, msg, iv)
    
    assert iv == bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    assert ct == bytes.fromhex("1fc519b6a47b3c562d905965e6e2f465")

test_enc_aes_cbc()

In [9]:
def dec_aes_cbc(k, ct, iv):
    """Dešifrira tajnopis s ključem in vrednostjo IV"""
    
    cipher = Cipher(algorithms.AES(k), modes.CBC(iv))
    dec = cipher.decryptor()
    
    pt = dec.update(ct) + dec.finalize()
    
    padder = padding.PKCS7(128).unpadder()
    pt = padder.update(pt) + padder.finalize()
    
    return pt

In [10]:
def test_dec_aes_cbc():  
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    ct = bytes.fromhex("1fc519b6a47b3c562d905965e6e2f465")
    
    pt = dec_aes_cbc(key, ct, iv)

    assert pt == b"Hello World!"

test_dec_aes_cbc()

## Naloga 3: Šifra enkratne podloge (OTP)

Implementirajte šifrirni in dešifrirni algorite enkratne podloge. Če uporabite rešitev iz prve naloge, bo implementacija trivialno kratka.

In [ ]:
def enc_otp(key, pt):
    """Sifrirni algoritem enkratne podloge"""
    pass

In [ ]:
assert enc_otp([1, 2, 3], [1, 1, 1]) == xor_bytes([1, 2, 3], [1, 1, 1])

In [ ]:
def dec_otp(key, ct):
    """Desifrirni algoritem enkratne podloge"""
    pass

In [ ]:
assert dec_otp([1, 2, 3], [1, 1, 1]) == xor_bytes([1, 2, 3], [1, 1, 1])

## Naloga 4: Šifriranje in dešifriranje

Uporabite metodi šifriranja in dešifriranja iz prejšnje naloge in zašifrirajte sporočilo `Enkratna podloga je popolno tajna šifra.` 

Implementacijo šifriranja podajte v telesu funkcije `example_enc()`. Za ključ uporabite vrednost `3d26ebcbc0b2ad0d15c6be1f6259fd89495451fc2245cd8dad40c480a87bbb3a7525a9ba4abb930417`. Pozor: podano vrednost ključa je potrebno prebrati kot seznam bajtov. To storite s pomočjo funkcije `bytes.fromhex(...)`.

In [ ]:
def example_enc():
    pass

example_enc()

In [ ]:
assert example_enc().hex() == "784880b9a1c6c36c35b6d17b0e369ae8693e34dc522abde2c12eaba0dc1ad15414056c1b23dde16539"

V telesu funkcije `example_dec()` dešifrirajte sporočilo `784880b9a1c6c36c35b6d17b0e369ae8693e34dc522abde2c12eaba0dc1ad15414056c1b23dde16539` in pri tem uporabite isti ključ kot ste ga pri šifriranju.

In [ ]:
def example_dec():
    pass

example_dec().decode("utf8")

In [ ]:
assert example_dec().decode("utf8") == "Enkratna podloga je popolno tajna šifra."

## Naloga 5: Napad na večkratno podlogo

Poglejmo, kaj lahko naredi napadalec, če isto podlogo uporabimo za šifriranje več sporočil.

### Naloga 5.1: Delno dešifriranje

Za začetek si pripravimo implementacijo dešifriranega algoritma, ki dešifrira tajnopis tudi, če nam kakšen del ključa manjka. Dešifrirani algoritem naj dešifrira kot običajno, edina izjema so znaki, ki je vrednost ključa enaka 0. V tem primeru, naj kot pripadajoč znak v tajnopisu nastavi vrednost `*`.

In [ ]:
def dec_otp_partial(key, ct):
    """Desifrira samo tiste znake, kjer kljuc ni 0 -- ce je, kot znak nastavi simbol *"""
    pass

dec_otp_partial(
    bytes.fromhex("0000ebcbc0b2ad0d15c6be1f6259fd89495451fc0000cd8dad40c480a87bbb3a7525a9ba4abb930000"),
    bytes.fromhex("784880b9a1c6c36c35b6d17b0e369ae8693e34dc522abde2c12eaba0dc1ad15414056c1b23dde16539")
).decode("utf8")

In [ ]:
assert dec_otp_partial([1, 0, 1], [2, 2, 2]) == bytes([3, ord("*"), 3])
assert dec_otp_partial([1, 1, 1], [2, 2, 2]) == dec_otp([1, 1, 1], [2, 2, 2])

### Tajnopisi

Tajnopisi so shranjeni v binarni obliki v datotekah `data/ct_i.bin`. Preberimo jih z diska.

In [ ]:
def load_cts():
    cipher_texts = []
    for i in range(10):
        with open(f"data/ct_{i}.bin", "rb") as h:
            cipher_texts.append(h.read())
    return cipher_texts

cipher_texts = load_cts()

print("Izpis prvih 50 bajtov tajnopisa šestnajstiško")
for i, c in enumerate(cipher_texts):
    print(f"Tajnopis {i}, dolžina {len(c)}:  {c.hex()[:100]}...")

Opazimo, da so vsi tajnopisi dolgi 109 bajtov. 

Vaša naloga je, da ugotovite ključ in z njim dešifrirajte sporočila. Pri tem lahko upoštevate še, da so sporočila v Slovenščini ter sestojijo le iz malih črk in presledkov: šumniki, števke in vsa ostala ločila so odstranjeni.

### Naloga 5.2: Funkcija `multiple_xor(pos, cts)`

Implementirajte funkcijo `multiple_xor(pos, cts)`, ki na vhodu prejme pozicijo znaka v tajnopisu in seznam tajnopisov. Metoda nato vrne bajt ključa oz. vrednost 0, če določi, da bajta v ključu na dani poziciji ni mogoče ugotoviti.

Namig: implementacija je dokaj kratka, če uporabite vgrajeno funkcijo `all`. V nasprotnem primeru boste potrebovali vgnezdeno zanko.

In [ ]:
def multiple_xor(pos, cts):
    pass


def test_multiple_xor():
    """Testira implementacijo funkcije multiple_xor
    
    Če ne vrne napake, je implementacija pravilna"""
    
    test_key = bytes.fromhex("cd04c883d62ca6e58f3cc3c554bc77")
    test_cts = [
      enc_otp(test_key, "v ponedeljek in".encode("ascii")),
      enc_otp(test_key, "ob uri na mestu".encode("ascii")),
      enc_otp(test_key, "zelo nenavadno ".encode("ascii"))
    ]

    assert multiple_xor(0, test_cts) == 0
    assert multiple_xor(1, test_cts) == test_key[1]
    assert multiple_xor(2, test_cts) == test_key[2]
    assert multiple_xor(3, test_cts) == 0
    assert multiple_xor(4, test_cts) == test_key[4]
    assert multiple_xor(5, test_cts) == 0
    assert multiple_xor(6, test_cts) == test_key[6]
    assert multiple_xor(7, test_cts) == 0
    assert multiple_xor(8, test_cts) == 0
    assert multiple_xor(9, test_cts) == test_key[9]
    assert multiple_xor(10, test_cts) == 0
    assert multiple_xor(11, test_cts) == 0
    assert multiple_xor(12, test_cts) == test_key[12]
    assert multiple_xor(13, test_cts) == 0
    assert multiple_xor(14, test_cts) == test_key[14]
    
test_multiple_xor()

### Naloga 5.3: Funkcija `guess_key(cts)`

Implementirajte funkcijo `guess_key(cts)`, ki na vhodu prejme seznam tajnopisov in nato vrne seznam bajtov, ki predstavljajo ključ, s katerim so bili tajnopisi ustvarjeni. V primeri, da kakšen bajt v ključu ni mogoče ugotoviti, nastavite njegovo vrednost na 0.

In [ ]:
def guess_key(ciphertexts):
    pass


gk = guess_key(cipher_texts)
for c in cipher_texts:
    print(dec_otp_partial(gk, c).decode("ascii"))

Nekatere bajte v ključih ugotovi algoritem, ostale pa poskusite dopolniti sami. 

## Naloga 6: Gnetljivost tajnopisa

Pri zadnji nalogi bomo kot napadalec spremenili tajnopis, tako da bo sprememba vidna v čistopisu. Zgodba je sledeča.

Ana želi poslati zaupno pošto Boru, vas, ki igrate vlogo napadalca Nandija, pa vsebina sporočila _zares_ zanima.

Na srečo vam gre nekaj reči na roko. 

Prvič, Anin računalnik nima internetne povezave, vaš mobilni telefon pa. Zato ji prijazno ponudite, da zanjo postavite mobilno dostopno točko, preko katere se bo lahko povezala v internet in dostopala do poštnega strežnika. Poštni strežnik bo nato sporočilo dostavil Boru. 

Ana in Bor uporabljata poseben protokol za pošto: protokol FMTP -- Funny Mail Transfer Protocol.  Gre za preprost besedilno-osnovan protokol: prva vrstica označuje naslov prejemnika, druga naslov pošiljatelja, tretja zadevo, četrta je vedno prazna in na koncu je sporočilo.

Vse, kar mora Anin poštni odjemalec storiti, da pošlje sporočilo, je, da strežniku FMTP dostavi besedilni niz, podoben naslednjemu.

```txt
prejemnik@enadomena.com
posiljatelj@drugadomena.com
Zadeva sporocila

<Samo sporocilo>
```

Vse morebitne predhodne ali zaključne presledke v vsaki vrstici poštni strežnik ignorira oz. odstrani pred obdelavo. Na primer, e-pošto zgoraj bi lahko napisali kot spodnjo in ne bi bilo nobene razlike.


```txt
                    prejemnik@enadomena.com               
    posiljatelj@drugadomena.com                    
Zadeva sporocila

<Samo sporocilo>
```

Tretjič, Ana vam v dobri veri -- kakšna naivnost! -- pove, da pošilja elektronsko sporočilo Boru in da uporablja protokol FMTP. (Torej poznate strukturo sporočila in vsebino prve vrstice čistopisa.)

Četrtič, Ana uporablja enkratno podlogo, mehanizmov za zagotavljanje celovitosti pa ni.

Ker nastopate v vlogi **posrednika**, lahko sedaj spremenite sporočilo tako, da strežnik FMTP ne bo poslal pošte Boru, temveč jo bo poslal na naslov, ki ga nadzirate vi, npr. `nandi@obvlada.si`.

In [ ]:
# Ana in Bor si enkrat tedensko v zivo izmenjata 1000 bajtov
# nakljucnih vrednosti za morebitne potrebe sifriranja
ana_bor_psk = gen_key(1000)

In [ ]:
# Ana pripravi sporočilo
email = """bor@student.uni-lj.si
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.""".strip()

# In ga šifrira z enkratno podlogo
ct = enc_otp(ana_bor_psk, email.encode("utf8"))

# Nato ga pošlje (to simuliramo) na streznik FMTP
print("Ana pošilja šifrirano sporočilo na strežnik")
print("Tajnopis (HEX):", ct.hex())

Sedaj je na potezi napadalec Nandi.

Implementirajte funckijo `change_ct(ct, new_email)`, ki na vhodu prejme tajnopis in email naslov. Spremenite tajnopis tako, da ga bo strežnik FMTP še vedno brez dešifriral, a kot prejemnik ne bo naveden Bor, temveč email naslov, ki je podan v argumentu `new_email`. Predpostavite lahko, da bo `new_email` vedno krajši ali enak od naslova `bor@student.uni-lj.si`.

In [ ]:
def change_ct(ct, new_email):
    pass

S poganjanjem spodnje celice lahko preverite, ali vaša funkcija deluje pravilno. 

In [ ]:
def fmtp_receive(key, ct):
    print("Streznik FMTP prejel sporocilo:", ct.hex())
    pt = dec_otp(key, ct)
    print("Desifrirano sporocilo:")
    print(pt.decode("utf8"))

fmtp_receive(ana_bor_psk, change_ct(ct, "nandi@obvlada.si"))